In [ ]:
安装模块 scanpy rdkit

In [ ]:
✅ 第 1 步：从 GSE210248 提取肺血管（PA）特异性基因评分 → 生成 pv_signature.csv

下载链接: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE210248

# 解压表达矩阵
tar -xzvf GSE210248_Human_PA_Integrated.tar.gz

# 解压元数据（如果需要）
gunzip GSE210248_Human_PA_Metadata.csv.gz

📌 操作目标：
解压并读取 GSE210248 数据
根据 metadata 识别 肺动脉内皮细胞（PA Endothelial） 和 肺动脉平滑肌细胞（PA SMC）
计算这些细胞 vs. 其他肺细胞的 log₂FC（连续评分）
输出：results/stage3/pv_signature.csv（gene_symbol, log2fc）

In [3]:
# code/stage4/0_extract_pv_signature.py
import scanpy as sc
import pandas as pd
import os

# 1. 加载元数据
meta = pd.read_csv("data/raw/step4/GSE210248_Human_PA_Metadata.csv", sep=';')
meta['cell_id'] = meta.iloc[:, 0]  # 第一列是 cell ID

# 2. 筛选肺动脉细胞 (SMC / EC)
pa_mask = meta['Cell_annotation'].str.contains(r'SMC|EC', case=True, na=False)
print(f"Found {pa_mask.sum()} PA cells.")

# 3. 读取 10x 格式矩阵
adata = sc.read_10x_mtx(
    "data/raw/step4/Human_PA_Integrated",
    var_names='gene_symbols',  # 使用基因名而非 Ensembl ID
    cache=True
)

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata  # 可选，保留原始计数

# 4. 对齐细胞：只保留 metadata 中存在的细胞
meta_cells = set(meta['cell_id'])
adata_cells = set(adata.obs_names)
common_cells = list(meta_cells & adata_cells)
print(f"Common cells: {len(common_cells)}")

adata = adata[common_cells, :].copy()

# 5. 添加分组标签
cell_to_is_pa = dict(zip(meta['cell_id'], pa_mask))
adata.obs['group'] = ['PA' if cell_to_is_pa.get(c, False) else 'Other' for c in adata.obs_names]

# 6. 差异表达分析
sc.tl.rank_genes_groups(adata, 'group', groups=['PA'], reference='Other', method='wilcoxon')

# 7. 提取结果
res = adata.uns['rank_genes_groups']
degs = pd.DataFrame({
    'gene_symbol': res['names']['PA'],
    'log2fc': res['logfoldchanges']['PA'],
    'pval': res['pvals']['PA'],
    'padj': res['pvals_adj']['PA']
})

# 8. 保存
os.makedirs("results/step4_outputs", exist_ok=True)
degs.to_csv("results/step4_outputs/pv_signature.csv", index=False)
print(f"✅ Saved pv_signature.csv with {len(degs)} genes.")

Found 3126 PA cells.
Common cells: 22704
✅ Saved pv_signature.csv with 19662 genes.


In [ ]:
构建带 肺血管特异性（pv_log2fc） 和 心脏核心功能（is_heart_core） 双特征的药-靶数据集

In [5]:
import pandas as pd
import os

# 1. 加载真实存在的输入文件
pv_sig = pd.read_csv("results/step4_outputs/pv_signature.csv")          # 刚生成的单细胞评分
drug_compounds = pd.read_csv("results/step3_outputs/druggable_targets_compounds.csv")  # 阶段 III 的药-靶对
heart_genes = set(pd.read_csv("results/step1_outputs/heart_core_function_genes.csv")["gene_symbol"])

# 2. 构建基因 → log2fc 映射
gene_to_log2fc = dict(zip(pv_sig["gene_symbol"], pv_sig["log2fc"]))

# 3. 添加特征
drug_compounds["pv_log2fc"] = drug_compounds["gene_symbol"].map(gene_to_log2fc).fillna(0.0)
drug_compounds["is_heart_core"] = drug_compounds["gene_symbol"].isin(heart_genes).astype(int)

# 4. （可选）只保留肺血管富集靶点（例如 log2fc > 0.5）
# filtered = drug_compounds[drug_compounds["pv_log2fc"] > 0.5].copy()
# 但建议先保留全部，后续在建模时加权

# 5. 保存
os.makedirs("results/step4_outputs", exist_ok=True)
drug_compounds.to_csv("results/step4_outputs/drug_target_features.csv", index=False)

print("✅ 已生成: results/step4_outputs/drug_target_features.csv")
print(f"   - 总记录数: {len(drug_compounds)}")
print(f"   - 心脏核心靶点数: {drug_compounds['is_heart_core'].sum()}")
print(f"   - 平均 pv_log2fc: {drug_compounds['pv_log2fc'].mean():.3f}")

✅ 已生成: results/step4_outputs/drug_target_features.csv
   - 总记录数: 2
   - 心脏核心靶点数: 2
   - 平均 pv_log2fc: 2.779


In [ ]:
虽然只有 2 个药-靶对，但：
✅ 全部属于 心脏核心功能基因
✅ 肺血管高度富集（平均 log₂FC ≈ 2.78，即 ～6.8 倍高表达）
这说明你筛选出的候选药物（如 Sampatrilat）同时作用于 HPH 的两个关键环节：
肺血管重塑（高 pv_log2fc）
右心功能维持（is_heart_core = 1）

In [ ]:
🔜 下一步：添加药物化学描述符（用于虚拟点云 X 轴）
将为这 2 个化合物提取 分子指纹或理化性质（如 MW, LogP, TPSA 等），来自 ChEMBL。

In [ ]:
使用这个数据库文件来添加药物描述符。
✅ 使用已有 data/raw/step1/chembl_36/chembl_36_sqlite/chembl_36.db 文件的脚本

In [12]:
import pandas as pd
import sqlite3
import os

# 1. 加载数据
df = pd.read_csv("results/step4_outputs/drug_target_features.csv")
chembl_ids = df["compound_chembl_id"].unique()

# 2. 连接数据库
db_path = "data/raw/step1/chembl_36/chembl_36_sqlite/chembl_36.db"
conn = sqlite3.connect(db_path)

# 3. 查询
query = """
SELECT 
    m.chembl_id,
    s.canonical_smiles,
    cp.full_mwt AS mol_weight,
    cp.alogp AS logp,
    cp.psa AS tpsa,
    cp.hbd AS hbd,
    cp.hba AS hba,
    cp.rtb AS num_rota,
    cp.qed_weighted AS qed_weighted
FROM molecule_dictionary m
JOIN compound_structures s ON m.molregno = s.molregno
JOIN compound_properties cp ON m.molregno = cp.molregno
WHERE m.chembl_id IN ({})
""".format(",".join(f"'{cid}'" for cid in chembl_ids))

props_df = pd.read_sql_query(query, conn)
conn.close()

# 4. 合并（关键修正！）
df_final = df.merge(
    props_df,
    left_on="compound_chembl_id",
    right_on="chembl_id",
    how="left"
).drop(columns=["chembl_id"])

# 5. 保存
os.makedirs("results/step4_outputs", exist_ok=True)
df_final.to_csv("results/step4_outputs/drug_target_features_with_desc.csv", index=False)

print("✅ 成功添加药物描述符！")

✅ 成功添加药物描述符！


In [ ]:
✅ 下一步：扩充 (Drug, Target) 候选对，构建一个包含 PV 和 RV 信号的平衡数据集（目标：300–500 对）
构建药理点云，需要 PV + RV 药物。
🧭 下一步：生成 心脏核心药物列表

In [1]:
# code/stage4/generate_heart_core_drugs.py
import pandas as pd
import sqlite3
import os

# 创建输出目录
os.makedirs("results/step4_outputs", exist_ok=True)

# 1. 加载心脏核心功能基因
heart_genes_df = pd.read_csv("results/step1_outputs/heart_core_function_genes.csv")
heart_genes = heart_genes_df["gene_symbol"].dropna().unique().tolist()
print(f"❤️ 将使用 {len(heart_genes)} 个心脏核心基因查询 ChEMBL:")
print(heart_genes[:10])  # 打印前10个

# 2. 连接 ChEMBL 数据库 —— 使用你提供的正确路径
db_path = "data/raw/step1/chembl_36/chembl_36_sqlite/chembl_36.db"
if not os.path.exists(db_path):
    raise FileNotFoundError(f"❌ ChEMBL 数据库未找到，请检查路径:\n{db_path}")

conn = sqlite3.connect(db_path)

# 3. 构建 SQL 查询
placeholders = ','.join('?' * len(heart_genes))
query = f"""
SELECT 
    t.chembl_id AS target_chembl_id,
    cs.component_synonym AS gene_symbol,
    md.chembl_id AS compound_chembl_id,
    md.pref_name AS compound_name,
    a.standard_value,
    a.standard_type,
    CASE WHEN md.max_phase >= 1 THEN 1 ELSE 0 END AS is_approved_drug,
    cs_smiles.canonical_smiles,
    a.pchembl_value
FROM activities a
JOIN assays ass ON a.assay_id = ass.assay_id
JOIN target_dictionary t ON ass.tid = t.tid
JOIN target_components tc ON t.tid = tc.tid
JOIN component_synonyms cs ON tc.component_id = cs.component_id
JOIN compound_records cr ON a.record_id = cr.record_id
JOIN molecule_dictionary md ON cr.molregno = md.molregno
JOIN compound_structures cs_smiles ON md.molregno = cs_smiles.molregno
WHERE 
    cs.component_synonym IN ({placeholders})
    AND cs.syn_type = 'GENE_SYMBOL'
    AND t.organism = 'Homo sapiens'
    AND a.pchembl_value >= 5.0
    AND cs_smiles.canonical_smiles IS NOT NULL
"""

# 4. 执行查询
print("⏳ 正在查询 ChEMBL，请稍候...")
try:
    heart_drugs = pd.read_sql_query(query, conn, params=heart_genes)
except Exception as e:
    print("❌ 查询出错:", e)
    heart_drugs = pd.DataFrame()

conn.close()

# 5. 去重 & 保存
if not heart_drugs.empty:
    heart_drugs = heart_drugs.drop_duplicates(subset=["compound_chembl_id", "gene_symbol"])
    print(f"✅ 找到 {len(heart_drugs)} 条高活性心脏核心靶向记录")
    
    # 检查是否包含 Sampatrilat 和 Aladorian
    names_upper = heart_drugs["compound_name"].str.upper().fillna("")
    has_sampatrilat = names_upper.str.contains("SAMPATRILAT").any()
    has_aladorian = names_upper.str.contains("ALADORIAN").any()
    print(f"🔍 包含 Sampatrilat: {has_sampatrilat}, Aladorian: {has_aladorian}")
else:
    print("⚠️ 未找到满足条件的心脏药物（极不可能）")

# 保存结果
output_file = "results/step4_outputs/heart_core_drugs.csv"
heart_drugs.to_csv(output_file, index=False)
print(f"💾 已保存至: {output_file}")

❤️ 将使用 19 个心脏核心基因查询 ChEMBL:
['NPPA', 'NPPB', 'TNNT2', 'RYR2', 'SCN5A', 'MYL3', 'HAND2', 'IRX4', 'MEF2C', 'NKX2-5']
⏳ 正在查询 ChEMBL，请稍候...
✅ 找到 1502 条高活性心脏核心靶向记录
🔍 包含 Sampatrilat: True, Aladorian: True
💾 已保存至: results/step4_outputs/heart_core_drugs.csv


In [ ]:
✅ 基于 19 个心脏核心功能基因，从 ChEMBL 中检索到 1502 条高活性（pChEMBL ≥ 5）、人源、有 SMILES 的药物-靶点记录
✅ 包含 Sampatrilat（NPPA 靶向）和 Aladorian（RYR2 靶向）——说明结果完整且一致

🧭 下一步：生成 肺核心药物列表

In [3]:
# code/stage3/generate_lung_core_drugs.py
import pandas as pd
import sqlite3
import os

os.makedirs("results/step3b_outputs", exist_ok=True)

# 1. 加载肺核心基因
lung_genes = pd.read_csv("results/step1_outputs/lung_core_function_genes.csv")["gene_symbol"].dropna().unique().tolist()
print(f"使用 {len(lung_genes)} 个肺核心基因查询 ChEMBL:")
print(lung_genes[:10])

# 2. 正确的数据库路径
db_path = "data/raw/step1/chembl_36/chembl_36_sqlite/chembl_36.db"
if not os.path.exists(db_path):
    raise FileNotFoundError(f"数据库未找到: {db_path}")

conn = sqlite3.connect(db_path)

# 3. 查询（与心脏完全相同的条件）
placeholders = ','.join('?' * len(lung_genes))
query = f"""
SELECT 
    t.chembl_id AS target_chembl_id,
    cs.component_synonym AS gene_symbol,
    md.chembl_id AS compound_chembl_id,
    md.pref_name AS compound_name,
    a.standard_value,
    a.standard_type,
    CASE WHEN md.max_phase >= 1 THEN 1 ELSE 0 END AS is_approved_drug,
    cs_smiles.canonical_smiles,
    a.pchembl_value
FROM activities a
JOIN assays ass ON a.assay_id = ass.assay_id
JOIN target_dictionary t ON ass.tid = t.tid
JOIN target_components tc ON t.tid = tc.tid
JOIN component_synonyms cs ON tc.component_id = cs.component_id
JOIN compound_records cr ON a.record_id = cr.record_id
JOIN molecule_dictionary md ON cr.molregno = md.molregno
JOIN compound_structures cs_smiles ON md.molregno = cs_smiles.molregno
WHERE 
    cs.component_synonym IN ({placeholders})
    AND cs.syn_type = 'GENE_SYMBOL'
    AND t.organism = 'Homo sapiens'
    AND a.pchembl_value >= 5.0
    AND cs_smiles.canonical_smiles IS NOT NULL
"""

print("⏳ 查询肺核心药物...")
lung_drugs = pd.read_sql_query(query, conn, params=lung_genes)
conn.close()

# 4. 去重 & 保存
lung_drugs = lung_drugs.drop_duplicates(subset=["compound_chembl_id", "gene_symbol"])
print(f"✅ 找到 {len(lung_drugs)} 条肺核心药物记录")

output_file = "results/step4_outputs/lung_core_drugs.csv"
lung_drugs.to_csv(output_file, index=False)
print(f"💾 保存至: {output_file}")

使用 16 个肺核心基因查询 ChEMBL:
['PDPN', 'SFTPB', 'LAMP3', 'HOPX', 'SCGB3A2', 'AGER', 'CAV1', 'SFTPC', 'SFTPA2', 'SFTPA1']
⏳ 查询肺核心药物...
✅ 找到 71 条肺核心药物记录
💾 保存至: results/step4_outputs/lung_core_drugs.csv


In [ ]:
✅ 在 16 个人工精选的肺核心功能基因中，有若干个被现有高活性化合物（pChEMBL ≥ 5）靶向，共获得 71 条药物-靶点记录。

🧩 下一步：合并肺和心脏药物，构建统一的“核心组织药物集”
读取两个文件
添加 tissue 列（"lung" 或 "heart"）
合并为一个 CSV，用于后续 分子指纹 & 相似性分析

In [4]:
# code/stage5/merge_tissue_core_drugs.py
import pandas as pd
import os

os.makedirs("results/step4_outputs", exist_ok=True)

# 1. 读取肺药物
lung_df = pd.read_csv("results/step4_outputs/lung_core_drugs.csv")
lung_df["tissue"] = "lung"
print(f"肺药物: {len(lung_df)} 条")

# 2. 读取心脏药物
heart_df = pd.read_csv("results/step4_outputs/heart_core_drugs.csv")
heart_df["tissue"] = "heart"
print(f"❤️ 心脏药物: {len(heart_df)} 条")

# 3. 合并
merged_df = pd.concat([lung_df, heart_df], ignore_index=True)
print(f"📊 合并后总计: {len(merged_df)} 条")

# 4. 只保留后续分析需要的列（可选，保持简洁）
required_cols = [
    'compound_chembl_id', 'compound_name', 'canonical_smiles',
    'gene_symbol', 'tissue', 'is_approved_drug', 'pchembl_value'
]
merged_df = merged_df[required_cols]

# 5. 去重（按 compound_chembl_id + tissue + gene_symbol）
merged_df = merged_df.drop_duplicates(
    subset=['compound_chembl_id', 'tissue', 'gene_symbol']
).reset_index(drop=True)

print(f"✅ 去重后: {len(merged_df)} 条")

# 6. 保存
output_file = "results/step4_outputs/core_tissue_drugs_merged.csv"
merged_df.to_csv(output_file, index=False)
print(f"💾 已保存至: {output_file}")

# 7. 简要统计
print("\n📈 统计概览:")
print(merged_df['tissue'].value_counts())
print("\n💊 已批准药物数量:")
print(merged_df.groupby('tissue')['is_approved_drug'].sum())

🫁 肺药物: 71 条
❤️ 心脏药物: 1502 条
📊 合并后总计: 1573 条
✅ 去重后: 1573 条
💾 已保存至: results/step4_outputs/core_tissue_drugs_merged.csv

📈 统计概览:
tissue
heart    1502
lung       71
Name: count, dtype: int64

💊 已批准药物数量:
tissue
heart    104
lung       1
Name: is_approved_drug, dtype: int64


In [ ]:
重复类型	                                 是否允许	       处理方式
完全重复（相同 Drug + Target）	        ❌ 不允许	    去重，保留最佳记录
近似重复（不同 Drug/Target 但坐标接近）	✅ 允许且有益	依靠 features + 模型容量 区分
一药多靶	                                ✅ 必须保留	    每个靶点是一个独立点

In [1]:
# code/stage4/inspect_merged_drugs.py
import pandas as pd

# 加载合并后的药物数据
df = pd.read_csv("results/step4_outputs/core_tissue_drugs_merged.csv")

print("🔍 文件基本信息:")
print(f"- 总记录数: {len(df)}")
print(f"- 列名: {list(df.columns)}\n")

print("🧬 按组织统计:")
print(df['tissue'].value_counts(), "\n")

print("💊 已批准药物统计:")
print(df.groupby('tissue')['is_approved_drug'].agg(['sum', 'count']), "\n")

print("🧪 示例记录（前3行）:")
print(df.head(3)[['compound_name', 'gene_symbol', 'tissue', 'canonical_smiles', 'is_approved_drug']], "\n")

print("⚠️ 检查潜在问题:")

# 检查 SMILES 是否为空
missing_smiles = df['canonical_smiles'].isnull().sum()
print(f"- 空 SMILES 数量: {missing_smiles}")

# 检查 tissue 是否只有 lung/heart
invalid_tissue = ~df['tissue'].isin(['lung', 'heart'])
print(f"- 非标准 tissue 标签数量: {invalid_tissue.sum()}")

# 检查重复 compound_chembl_id + tissue + gene_symbol
dups = df.duplicated(subset=['compound_chembl_id', 'tissue', 'gene_symbol']).sum()
print(f"- 重复记录数: {dups}")

# 查看肺中唯一的已批准药物是谁？
lung_approved = df[(df['tissue'] == 'lung') & (df['is_approved_drug'] == 1)]
if not lung_approved.empty:
    print("\n🫁 肺中已批准药物:")
    print(lung_approved[['compound_name', 'gene_symbol']])
else:
    print("\n🫁 肺中无已批准药物")

🔍 文件基本信息:
- 总记录数: 1573
- 列名: ['compound_chembl_id', 'compound_name', 'canonical_smiles', 'gene_symbol', 'tissue', 'is_approved_drug', 'pchembl_value']

🧬 按组织统计:
tissue
heart    1502
lung       71
Name: count, dtype: int64 

💊 已批准药物统计:
        sum  count
tissue            
heart   104   1502
lung      1     71 

🧪 示例记录（前3行）:
  compound_name gene_symbol tissue  \
0           NaN        AGER   lung   
1           NaN        AGER   lung   
2           NaN        AGER   lung   

                                    canonical_smiles  is_approved_drug  
0  CCN(CC)CCOc1ccccc1Nc1nc(-c2ccc(Cl)cc2)cc(-c2cc...                 0  
1  CCCCc1sc(-c2ccc(OCCN(CC)CC)cc2)nc1-c1ccc(Oc2cc...                 0  
2  CCCCc1sc(-c2ccc(OCCN(C)C)cc2)nc1-c1ccc(Oc2ccc(...                 0   

⚠️ 检查潜在问题:
- 空 SMILES 数量: 0
- 非标准 tissue 标签数量: 0
- 重复记录数: 0

🫁 肺中已批准药物:
   compound_name gene_symbol
24    AZELIRAGON        AGER
